In [1]:
from datasets import load_dataset

ds = load_dataset("bespokelabs/Bespoke-Stratos-35k")

/home/heegyu/miniconda/envs/heegyu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 35028/35028 [00:01<00:00, 18320.10 examples/s]


In [3]:
system_message = ds['train'][0]['system']
print(system_message)

Your role as an assistant involves thoroughly exploring questions through a systematic long thinking process before providing the final precise and accurate solutions. This requires engaging in a comprehensive cycle of analysis, summarizing, exploration, reassessment, reflection, backtracing, and iteration to develop well-considered thinking process. Please structure your response into two main sections: Thought and Solution. In the Thought section, detail your reasoning process using the specified format: <|begin_of_thought|> {thought with steps separated with '\n\n'} <|end_of_thought|> Each step should include detailed considerations such as analisying questions, summarizing relevant findings, brainstorming new ideas, verifying the accuracy of the current steps, refining any errors, and revisiting previous steps. In the Solution section, based on various attempts, explorations, and reflections from the Thought section, systematically present the final solution that you deem correct. 

In [5]:
def change_to_messages(item):
    messages = []

    messages.append({
        "role": "system",
        "content": item["system"]
    })
    for conv in item["conversations"]:
        messages.append({
            "role": conv["from"],
            "content": conv["value"]
        })

    return {"messages": messages}

ds = ds.map(change_to_messages, remove_columns=ds['train'].column_names)
ds.push_to_hub("heegyu/Bespoke-Stratos-35k-messages")

Map:   0%|          | 0/35028 [00:00<?, ? examples/s]

Uploading the dataset shards: 100%|██████████| 2/2 [01:31<00:00, 45.76s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/heegyu/Bespoke-Stratos-35k-messages/commit/22bdb4b91d5c70f7b95ccfca999522bef9d4cb2f', commit_message='Upload dataset', commit_description='', oid='22bdb4b91d5c70f7b95ccfca999522bef9d4cb2f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/heegyu/Bespoke-Stratos-35k-messages', endpoint='https://huggingface.co', repo_type='dataset', repo_id='heegyu/Bespoke-Stratos-35k-messages'), pr_revision=None, pr_num=None)

In [6]:
openo1sft = load_dataset("O1-OPEN/OpenO1-SFT")


Generating train split: 100%|██████████| 77685/77685 [00:01<00:00, 65634.40 examples/s]


In [19]:
from pprint import pprint

def map_openo1_sft(item):
    output = item["output"]
    output = output.replace("<Thought>", "<|begin_of_thought|>")
    output = output.replace("</Thought>", "<|end_of_thought|>")
    output = output.replace("<Output>", "<|begin_of_solution|>")
    output = output.replace("</Output>", "<|end_of_solution|>")
    
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": item["instruction"]
            },
            {
                "role": "assistant",
                "content": output
            }
        ]
    }

pprint(map_openo1_sft(openo1sft['train'][0]))

openo1sft = openo1sft.map(map_openo1_sft, remove_columns=openo1sft['train'].column_names)
openo1sft.push_to_hub("heegyu/OpenO1-SFT-35k-messages-bespoke")

{'messages': [{'content': 'Your role as an assistant involves thoroughly '
                          'exploring questions through a systematic long '
                          'thinking process before providing the final precise '
                          'and accurate solutions. This requires engaging in a '
                          'comprehensive cycle of analysis, summarizing, '
                          'exploration, reassessment, reflection, backtracing, '
                          'and iteration to develop well-considered thinking '
                          'process. Please structure your response into two '
                          'main sections: Thought and Solution. In the Thought '
                          'section, detail your reasoning process using the '
                          'specified format: <|begin_of_thought|> {thought '
                          "with steps separated with '\\n\\n'} "
                          '<|end_of_thought|> Each step should include '
 

Uploading the dataset shards: 100%|██████████| 1/1 [01:01<00:00, 62.00s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/heegyu/OpenO1-SFT-35k-messages-bespoke/commit/978368419b09158f4c2fe1110d1dc603be78a153', commit_message='Upload dataset', commit_description='', oid='978368419b09158f4c2fe1110d1dc603be78a153', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/heegyu/OpenO1-SFT-35k-messages-bespoke', endpoint='https://huggingface.co', repo_type='dataset', repo_id='heegyu/OpenO1-SFT-35k-messages-bespoke'), pr_revision=None, pr_num=None)

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-1.5B")

In [20]:
def count_tokens(messages):
    tokens = tokenizer.apply_chat_template(messages)
    return len(tokens)

# count_tokens(map_openo1_sft(openo1sft['train'][0])["messages"])

# df = ds['train'].to_pandas()
df = openo1sft['train'].to_pandas()

In [18]:
num_tokens = df["messages"].sample(1000).apply(count_tokens)
num_tokens.describe()

count     1000.000000
mean      5794.357000
std       5006.488241
min        764.000000
25%       2293.750000
50%       3925.000000
75%       7541.500000
max      27131.000000
Name: messages, dtype: float64